In [4]:
#!pip install bs4
#!pip install requests

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

# WebScrapping 

I used a wikipedia page to extract the post code and others data 

In [5]:
url3 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [6]:
data  = requests.get(url3).text
soup = BeautifulSoup(data,'html5lib')
table = soup.find('table')

Remove all of the post code which have been unassigned

In [7]:
name = []
code = []
for tds in table.findAll('td'):
    a = tds.text.strip()
    if a[3:] != 'Not assigned':
        code.append(a[:3])
        name.append(a[3:])

### Borough and Neighborhood are still together 

In [38]:
d = {'Code': code, 'Borough': name}
toronto= pd.DataFrame(data = d, index=None, columns=['Code','Borough'])
toronto.head(12)

,Code,Borough
0,M3A,North York(Parkwoods)
1,M4A,North York(Victoria Village)
2,M5A,Downtown Toronto(Regent Park / Harbourfront)
3,M6A,North York(Lawrence Manor / Lawrence Heights)
4,M7A,Queen's Park(Ontario Provincial Government)
5,M9A,Etobicoke(Islington Avenue)
6,M1B,Scarborough(Malvern / Rouge)
7,M3B,North York(Don Mills)North
8,M4B,East York(Parkview Hill / Woodbine Gardens)
9,M5B,"Downtown Toronto(Garden District, Ryerson)"


### splitting borough and neighborhood

neigh = []
newcode = []
newbor = []
for n,c in zip(name, code):
    if n.find('(')!= -1:
        #neighborough is present
        idx = n.find('(')
        idx2 = n.find(')')
        nghd = n[idx+1:idx2].split(' / ')
        neigh.extend(nghd)
        newcode.extend(len(nghd)*[c])
        newbor.extend(len(nghd)*[n[:idx]])

In [39]:
neigh = []
newcode = []
newbor = []
for n,c in zip(name, code):
    if n.find('(')!= -1:
        #neighborough is present
        idx = n.find('(')
        idx2 = n.find(')')
        nghd = n[idx+1:idx2].split(' / ')
        neigh.append(' '.join([str(elem+',') for elem in nghd])[:-1])
        newcode.extend([c])
        newbor.extend([n[:idx]])

In [40]:
d = {'PostalCode': newcode, 'Borough': newbor, 'Neighborhood': neigh}
toronto= pd.DataFrame(data = d, index=None, columns=['PostalCode','Borough','Neighborhood'])
toronto.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [41]:
toronto.shape

(103, 3)

In [42]:
#!pip install pgeocode
import pgeocode
nomi = pgeocode.Nominatim('ca')
lat = []
long = []
for nc in newcode:
    postal_code = nc
    location = nomi.query_postal_code(postal_code)
    latitude = location.latitude
    longitude = location.longitude
    #print(latitude, longitude)
    lat.append(latitude)
    long.append(longitude)

In [45]:
toronto['latitude'] = lat
toronto['longitude'] = long
toronto.head(11)

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills,43.7450,-79.3590
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
